# RNN을 이용한 텍스트 생성(One-to-many 구조)
'경마장에 있는 말이 뛰고 있다‘

'그의 말이 법이다‘

'가는 말이 고와야 오는 말이 곱다'

In [2]:
import numpy as np
import tensorflow as tf
seed = 2021
np.random.seed(seed)
tf.random.set_seed(seed)

In [1]:
text = '''
경마장에 있는 말이 뛰고 있다\n
그의 말이 법이다\n
가는 말이 고와야 오는 말이 곱다\n
'''

## 텍스트 전처리

In [3]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

In [4]:
# 단어 집합 생성
t = Tokenizer()
t.fit_on_texts([text])
# 케라스 Tokenizer은 1부터 시작

In [5]:
t.word_index

{'말이': 1,
 '경마장에': 2,
 '있는': 3,
 '뛰고': 4,
 '있다': 5,
 '그의': 6,
 '법이다': 7,
 '가는': 8,
 '고와야': 9,
 '오는': 10,
 '곱다': 11}

In [32]:
# 단어 집합 크기 설정
vocab_size = len(t.word_index) + 1
# 케라스 토크나이저의 정수 인코딩은 인덱스가 1부터 시작하지만,
# 케라스 원-핫 인코딩에서 배열의 인덱스가 0부터 시작하기 때문에
# 배열의 크기를 실제 단어 집합의 크기보다 +1로 생성해야함 
print('단어 집합의 크기 : %d' % vocab_size)

단어 집합의 크기 : 12


In [9]:
t.texts_to_sequences(['그의 말이 법이다'])

[[6, 1, 7]]

In [11]:
# 학습에 사용될 샘플 시퀀스
sequences = []
for line in text.split('\n'):   # # \n을 기준으로 문장 토큰화
    encoded = t.texts_to_sequences([line])[0]   #바깥쪽에 있는 리스트 제외
    for i in range(1, len(encoded)):
        sequence = encoded[:i+1]
        sequences.append(sequence)
sequences
# [0,0,0,0,8,1]
# 00000->패딩(x를 0으로 채움) 8,1->y값

[[2, 3],
 [2, 3, 1],
 [2, 3, 1, 4],
 [2, 3, 1, 4, 5],
 [6, 1],
 [6, 1, 7],
 [8, 1],
 [8, 1, 9],
 [8, 1, 9, 10],
 [8, 1, 9, 10, 1],
 [8, 1, 9, 10, 1, 11]]

In [12]:
max_len = max(len(s) for s in sequences)
max_len

6

## Padding
- 전체 샘플의 길이를 가장 긴 샘플의 길이(6) 패딩
- 'pre' 옵션을 주면 앞을 0으로 채움

In [13]:
# 전체 샘플의 길이를 6(가장 긴 샘플의 길이)으로 패딩
# 'pre' 옵션을 주면 앞을 0으로 패딩
sequences = pad_sequences(sequences, maxlen=max_len, padding='pre')
sequences[:5]

array([[0, 0, 0, 0, 2, 3],
       [0, 0, 0, 2, 3, 1],
       [0, 0, 2, 3, 1, 4],
       [0, 2, 3, 1, 4, 5],
       [0, 0, 0, 0, 6, 1]])

In [14]:
X = sequences[:,:-1]
y = sequences[:,-1]

In [33]:
# 레이블 데이터 y에 대해서 원-핫 인코딩을 수행
Y = to_categorical(y, num_classes=vocab_size)
Y[:3]

array([[0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.]], dtype=float32)

## 모델 정의
- Embedding
- SimpleRNN

In [17]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense

In [34]:
# Embedding=4, SimpleRNN=32
model = Sequential()
model.add(Embedding(vocab_size, 4, input_length=max_len-1))
model.add(SimpleRNN(32))
model.add(Dense(vocab_size, activation='softmax'))
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 5, 4)              48        
_________________________________________________________________
simple_rnn_1 (SimpleRNN)     (None, 32)                1184      
_________________________________________________________________
dense_1 (Dense)              (None, 12)                396       
Total params: 1,628
Trainable params: 1,628
Non-trainable params: 0
_________________________________________________________________


## 모델 설정/학습

In [35]:
model.compile(loss='categorical_crossentropy', 
              optimizer='adam', metrics=['accuracy'])

In [36]:
history = model.fit(X, Y, epochs=200, verbose=0)

history.history['accuracy'][-1]

0.9090909361839294

## 모델 검증

In [37]:
# 모델 검증용 문장을 생성하는 함수
def sentence_generation(model, t, current_word, n): # 모델, 토크나이저, 현재 단어, 반복할 횟수
    init_word = current_word # 처음 들어온 단어도 마지막에 같이 출력하기위해 저장
    sentence = ''
    for _ in range(n): # n번 반복
        encoded = t.texts_to_sequences([current_word])[0] # 현재 단어에 대한 정수 인코딩
        encoded = pad_sequences([encoded], maxlen=5, padding='pre') # 데이터에 대한 패딩
        #result = model.predict_classes(encoded, verbose=0)
        result = np.argmax(model.predict(encoded), axis=-1)
    # 입력한 X(현재 단어)에 대해서 Y를 예측하고 Y(예측한 단어)를 result에 저장.
        for word, index in t.word_index.items(): 
            if index == result: # 만약 예측한 단어와 인덱스와 동일한 단어가 있다면
                break # 해당 단어가 예측 단어이므로 break
        current_word = current_word + ' ' + word # 현재 단어 + ' ' + 예측 단어를 현재 단어로 변경
        sentence = sentence + ' ' + word # 예측 단어를 문장에 저장
        
    # for문이므로 이 행동을 다시 반복
    sentence = init_word + sentence
    return sentence

In [38]:
sentence_generation(model, t, '경마장에', 3)

'경마장에 말이 고와야 오는'

In [39]:
sentence_generation(model, t, '그의', 2)

'그의 말이 법이다'

## sentence_generation() 함수의 풀이 과정

In [40]:
enc = t.texts_to_sequences(['경마장에 말이 고와야'])[0]
enc = pad_sequences([enc], maxlen=5, padding='pre')
enc

array([[0, 0, 2, 1, 9]])

In [41]:
result = np.argmax(model.predict(enc), axis=-1)
result

array([10], dtype=int64)

In [42]:
for word, index in t.word_index.items(): 
    if index == result: 
        break 
print(word)

오는


## 모델 변화
- Embedding Vector 갯수: [2, 4, 6, 8]
- RNN 노드 갯수: [24, 32]

In [43]:
def execute_model(n_embed, n_rnn):
    model = Sequential()
    model.add(Embedding(vocab_size, n_embed, input_length=max_len-1))
    model.add(SimpleRNN(n_rnn))
    model.add(Dense(vocab_size, activation='softmax'))

    model.compile(loss='categorical_crossentropy', 
                  optimizer='adam', metrics=['accuracy'])
    history = model.fit(X, Y, epochs=200, verbose=0)
    print(f"Accuracy: {history.history['accuracy'][-1]:.4f}")

    print(sentence_generation(model, t, '경마장에', 3))
    print(sentence_generation(model, t, '그의', 2))
    print(sentence_generation(model, t, '가는', 5))

In [44]:
for n_embed in [2,4,6,8]:
    for n_rnn in [24,32]:
        print('===================================')
        print(f'n_embed={n_embed}, n_rnn={n_rnn}')
        execute_model(n_embed, n_rnn)
        print()

n_embed=2, n_rnn=24
Accuracy: 0.7273
경마장에 말이 말이 뛰고
그의 말이 말이
가는 말이 고와야 오는 말이 곱다

n_embed=2, n_rnn=32
Accuracy: 0.9091
경마장에 말이 말이 뛰고
그의 말이 법이다
가는 말이 고와야 오는 말이 곱다

n_embed=4, n_rnn=24
Accuracy: 0.9091
경마장에 말이 말이 뛰고
그의 말이 법이다
가는 말이 고와야 오는 말이 곱다

n_embed=4, n_rnn=32
Accuracy: 1.0000
경마장에 있는 말이 뛰고
그의 말이 법이다
가는 말이 고와야 오는 말이 곱다

n_embed=6, n_rnn=24
Accuracy: 0.9091
경마장에 말이 법이다 있다
그의 말이 법이다
가는 말이 고와야 오는 말이 곱다

n_embed=6, n_rnn=32
Accuracy: 1.0000
경마장에 있는 말이 뛰고
그의 말이 법이다
가는 말이 고와야 오는 말이 곱다

n_embed=8, n_rnn=24
Accuracy: 1.0000
경마장에 있는 말이 뛰고
그의 말이 법이다
가는 말이 고와야 오는 말이 곱다

n_embed=8, n_rnn=32
Accuracy: 0.9091
경마장에 말이 고와야 오는
그의 말이 법이다
가는 말이 고와야 오는 말이 곱다



embedding vector가 작아지면 자기 역할을 못하고 충분히 커져야함.